In [1]:
from manim import *
import numpy as np
from scipy.integrate import solve_ivp

## 1-Body Problem

I used __ChatGPT__ to speed up development of this.  Though it's worth noting it did make a lot of mistakes I had to fix, like rendering the path as a solid object, not rendering the earth, etc.

In [16]:
%%manim -v WARNING -qm EarthOrbitScene

# Constants
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
M_sun = 1.989e30  # Mass of the Sun (kg)
AU = 1.496e11  # Astronomical Unit (m)
year = 365.25 * 24 * 3600  # Seconds in a year

# Differential equation for Earth's orbit in 3D
def orbit_equations_3d(t, state):
    x, y, z, vx, vy, vz = state
    r_squared = x**2 + y**2 + z**2
    r = np.sqrt(r_squared)
    acceleration_factor = G * M_sun / r_squared
    ax = -acceleration_factor * x / r
    ay = -acceleration_factor * y / r
    az = -acceleration_factor * z / r
    return [vx, vy, vz, ax, ay, az]

# Initial conditions
x0 = AU
y0 = 0
z0 = 0
vx0 = 0
vy0 = 29.78e3
vz0 = 1.0e3

initial_conditions = [x0, y0, z0, vx0, vy0, vz0]

# Time span: 5 years
t_span = (0, 5 * year)
t_eval = np.linspace(0, 5 * year, 5000)

# Solve the ODE
solution = solve_ivp(orbit_equations_3d, t_span, initial_conditions, t_eval=t_eval)

# Normalize coordinates for visualization (scale down by AU for readability)
orbit_points_3d = np.vstack((solution.y[0], solution.y[1], solution.y[2])).T / AU

class EarthOrbitScene(ThreeDScene):
    def construct(self):
        # Set up 3D axes
        axes = ThreeDAxes(
            x_range=[-1.5, 1.5, 0.5],
            y_range=[-1.5, 1.5, 0.5],
            z_range=[-0.5, 0.5, 0.2],
            x_length=10,
            y_length=10,
            z_length=5,
        )

        # Add the Sun (yellow sphere at the origin)
        # used a dot instead of sphere because sphere doesn't become solid color by default
        sun = Dot3D(radius=0.1, color=YELLOW).move_to(ORIGIN)

        # Create the Earth (blue sphere)
        earth = Dot3D(radius=0.05, color=BLUE)

        # Add axes, Sun, and Earth to the scene
        self.add(axes, sun, earth)

        # Set the camera position
        self.set_camera_orientation(phi=75 * DEGREES, theta=45 * DEGREES, zoom=6)
    
        # Animate Earth's motion along the orbit
        orbit_path = VMobject(color=BLUE)
        orbit_path.set_points_as_corners([np.array([x, y, z]) for x, y, z in orbit_points_3d])

        time_tracker = ValueTracker(0)
        
        # Animate Earth following the orbit
        def update_earth(earth):
            t = int(time_tracker.get_value()) % len(orbit_points_3d)
            pos = orbit_points_3d[int(t)]
            earth.move_to(np.array([pos[0], pos[1], pos[2]]))
        earth.add_updater(update_earth)

        # Play the animation
        self.play(time_tracker.animate.set_value(len(orbit_points_3d) - 1), run_time=20, rate_func=linear)

Manim Community v0.18.1

## Two-Body Problem

This was created by __ChatGPT__ (with some fixes by me) based on pasting in my code above and asking it to make it a two-body problem with Earth and Mars but no sun.

I changed the conditions to match Pluto and Cheron instead of Earth and Mars to get a better simulation.  I didn't bother to fix variable names (eg. orbital_period)

__NOTE__: the simulation is unstable (orbit moves and decays) because scipy apparently isn't good at this kind of simulation (numerical errors build up)

__NOTE__: 3blue1brown just uses an elliptical path in his simulation - not the differential equations (even though he shows the Newtonian law on screen)

In [38]:
%%manim -v WARNING -qm TwoBodySimulation

# Constants
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
M_earth = 1.309e22  # Mass of the Earth (kg)
M_mars = 1.586e21  # Mass of Mars (kg)
distance = 19_640_000
AU = 1.496e11  # Astronomical Unit (m)
year = 365.25 * 24 * 3600  # Seconds in a year

# Differential equations for the two-body system
def two_body_equations(t, state):
    """
    state = [x_e, y_e, z_e, vx_e, vy_e, vz_e, x_m, y_m, z_m, vx_m, vy_m, vz_m]
    """
    # Unpack state
    x_e, y_e, z_e, vx_e, vy_e, vz_e, x_m, y_m, z_m, vx_m, vy_m, vz_m = state

    # Distances
    r_em_squared = (x_m - x_e)**2 + (y_m - y_e)**2 + (z_m - z_e)**2
    r_em = np.sqrt(r_em_squared)

    # Gravitational accelerations
    ax_e = G * M_mars * (x_m - x_e) / r_em_squared
    ay_e = G * M_mars * (y_m - y_e) / r_em_squared
    az_e = G * M_mars * (z_m - z_e) / r_em_squared

    ax_m = -G * M_earth * (x_m - x_e) / r_em_squared
    ay_m = -G * M_earth * (y_m - y_e) / r_em_squared
    az_m = -G * M_earth * (z_m - z_e) / r_em_squared

    return [
        vx_e, vy_e, vz_e,  # Earth's velocity components
        ax_e, ay_e, az_e,  # Earth's acceleration components
        vx_m, vy_m, vz_m,  # Mars's velocity components
        ax_m, ay_m, az_m,  # Mars's acceleration components
    ]


# Compute positions relative to the center of mass
r_earth = (M_mars / (M_earth+M_mars)) * distance
r_mars = (M_earth / (M_earth + M_mars)) * distance

# Compute velocities for circular motion
v_earth = np.sqrt(G * M_mars / distance)
v_mars = np.sqrt(G * M_earth / distance)

# Initial positions and velocities
initial_conditions = [
    -r_earth, 0, 0, 0, v_earth, 0,  # Pluto
    r_mars, 0, 0, 0, -v_mars, 0,  # Charon
]

# Time span
orbital_period = 6.387 * 24 * 3600  # Orbital period in seconds
t_span = (0, orbital_period / 1000)
t_eval = np.linspace(0, orbital_period / 1000, 2000000)  # High resolution

# Solve the ODE
solution = solve_ivp(
    two_body_equations, t_span, initial_conditions, t_eval=t_eval, 
    method="Radau",  
    rtol=1e-10,  # Adjust relative tolerance for higher precision
    atol=1e-12   # Adjust absolute tolerance for higher precision
)

# Extract the trajectories
earth_trajectory = np.vstack((solution.y[0], solution.y[1], solution.y[2])).T / 1e6  # Convert to km
mars_trajectory = np.vstack((solution.y[6], solution.y[7], solution.y[8])).T / 1e6  # Convert to km

# Combine both trajectories to find the overall maximum extent
all_trajectories = np.vstack((earth_trajectory, mars_trajectory))
max_extent = np.max(np.abs(all_trajectories))  # Maximum absolute extent in x, y, z

# Define the desired axis limit (e.g., axes will range from -1 to 1)
desired_axis_limit = 1  # Half-range for normalization
scale_factor = desired_axis_limit / max_extent

# Normalize the trajectories
earth_trajectory_normalized = earth_trajectory * scale_factor
mars_trajectory_normalized = mars_trajectory * scale_factor

# Radii for visualization
R_earth = scale_factor * 1188 / 1e3  # Radius in km
R_mars = scale_factor * 606 / 1e3  # Radius in km

class TwoBodySimulation(ThreeDScene):
    def construct(self):
        # Set up 3D axes
        axes = ThreeDAxes(
            x_range=[-1, 1, 0.5],
            y_range=[-1, 1, 0.5],
            z_range=[-1, 1, 0.5],
            x_length=10,
            y_length=10,
            z_length=10,
        )
        axes.center()
        
        # Create Earth and Mars
        earth = Dot3D(radius=R_earth, color=BLUE)
        mars = Dot3D(radius=R_mars, color=RED)

        # Add axes, Earth, and Mars to the scene
        self.add(axes, earth, mars)

        # Set the camera position
        self.set_camera_orientation(phi=75 * DEGREES, theta=45 * DEGREES, zoom=5)

        # Path for Earth and Mars
        earth_path = VMobject(color=BLUE)
        mars_path = VMobject(color=RED)

        earth_path.set_points_as_corners(
            axes.coords_to_point([np.array([x, y, z]) for x, y, z in earth_trajectory_normalized])
        )
        mars_path.set_points_as_corners(
            axes.coords_to_point([np.array([x, y, z]) for x, y, z in mars_trajectory_normalized])
        )

        # Time tracker
        time_tracker = ValueTracker(0)

        # Updaters for Earth and Mars
        def update_earth(mob):
            t = int(time_tracker.get_value()) % len(earth_trajectory_normalized)
            pos = earth_trajectory_normalized[t]
            mob.move_to(np.array([pos[0], pos[1], pos[2]]))

        def update_mars(mob):
            t = int(time_tracker.get_value()) % len(mars_trajectory_normalized)
            pos = mars_trajectory_normalized[t]
            mob.move_to(np.array([pos[0], pos[1], pos[2]]))

        earth.add_updater(update_earth)
        mars.add_updater(update_mars)

        # Animate the motion
        self.play(time_tracker.animate.set_value(len(earth_trajectory) - 1), run_time=20, rate_func=linear)

Manim Community v0.18.1

## Two-Body Problem Using Rebound Library

This uses a type of integration that is more suited to cyclical problems like gravity.

For some reason, the sizing and timing is totally different from above, which I had to manually figure out.  Also, in this simulation, Earth is moving in an almost straight line while Mars stays in orbit of it and moves along - I don't know why the above simulation missed that, but it is accurate, because the __center of mass__ is the fixed point in a two-body problems.

__NOTE__: the movement of the system is probably why the scaling changed since I scale it with the largest measurement.

__NOTE__: you have to enlarge the video to fullscreen to see the planets - they're a bit small

In [4]:
!pip install rebound

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.8/256.8 kB 4.1 MB/s eta 0:00:00a 0:00:01


In [51]:
%%manim -v WARNING -qm ReboundSimulation

import rebound

# Constants
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
M_earth = 1.309e22  # Mass of the Earth (kg)
M_mars = 1.586e21  # Mass of Mars (kg)
distance = 19_640_000
AU = 1.496e11  # Astronomical Unit (m)
year = 365.25 * 24 * 3600  # Seconds in a year

# Compute positions relative to the center of mass
r_earth = (M_mars / (M_earth+M_mars)) * distance
r_mars = (M_earth / (M_earth + M_mars)) * distance

# Compute velocities for circular motion  (something went wrong here)
v_earth = np.sqrt(G * M_mars / distance)
v_mars = np.sqrt(G * M_earth / distance)

# Initial positions and velocities
initial_conditions = [
    -r_earth, 0, 0, 0, v_earth, 0,  # Pluto
    r_mars, 0, 0, 0, -v_mars, 0,  # Charon
]

# Time span
orbital_period = 6.387 * 24 * 3600  # Orbital period in seconds
t_span = (0, 30*orbital_period)
t_eval = np.linspace(0, 30*orbital_period, 1000)  # Less resolution than for scipy (to avoid issues with rebound)

# Initialize Rebound simulation
sim = rebound.Simulation()
sim.G = G  # Use the gravitational constant

# Add Pluto and Charon using initial_conditions
sim.add(
    m=M_earth,
    x=initial_conditions[0],
    y=initial_conditions[1],
    z=initial_conditions[2],
    vx=initial_conditions[3],
    vy=initial_conditions[4],
    vz=initial_conditions[5],
)  # Pluto

sim.add(
    m=M_mars,
    x=initial_conditions[6],
    y=initial_conditions[7],
    z=initial_conditions[8],
    vx=initial_conditions[9],
    vy=initial_conditions[10],
    vz=initial_conditions[11],
)  # Charon

# Set up the symplectic integrator and timestep
sim.integrator = "whfast"  # Symplectic integrator
sim.dt = 30*orbital_period/1000

# Simulate and collect positions
earth_trajectory = []
mars_trajectory = []

for t in t_eval:
    sim.integrate(t)  # Advance to the next time step
    pluto = sim.particles[0]
    charon = sim.particles[1]
    earth_trajectory.append([pluto.x/1e6, pluto.y/1e6, pluto.z/1e6])
    mars_trajectory.append([charon.x/1e6, charon.y/1e6, charon.z/1e6])

# Combine both trajectories to find the overall maximum extent
all_trajectories = np.vstack((earth_trajectory, mars_trajectory))
max_extent = np.max(np.abs(all_trajectories))  # Maximum absolute extent in x, y, z

# Define the desired axis limit (e.g., axes will range from -1 to 1)
desired_axis_limit = 1  # Half-range for normalization
scale_factor = desired_axis_limit / max_extent

# Normalize the trajectories
earth_trajectory_normalized = np.array(earth_trajectory) * scale_factor
mars_trajectory_normalized = np.array(mars_trajectory) * scale_factor

# Radii for visualization
R_earth = scale_factor * 1188 / 1e3  # Radius in km
R_mars = scale_factor * 606 / 1e3  # Radius in km

class ReboundSimulation(ThreeDScene):
    def construct(self):
        # Set up 3D axes
        axes = ThreeDAxes(
            x_range=[-1, 1, 0.5],
            y_range=[-1, 1, 0.5],
            z_range=[-1, 1, 0.5],
            x_length=10,
            y_length=10,
            z_length=10,
        )
        axes.center()
        
        # Create Earth and Mars
        earth = Dot3D(radius=R_earth, color=BLUE)
        mars = Dot3D(radius=R_mars, color=RED)

        # Add axes, Earth, and Mars to the scene
        self.add(axes, earth, mars)

        # Set the camera position
        self.set_camera_orientation(phi=75 * DEGREES, theta=45 * DEGREES, zoom=5)

        # Path for Earth and Mars
        earth_path = VMobject(color=BLUE)
        mars_path = VMobject(color=RED)

        earth_path.set_points_as_corners(
            axes.coords_to_point([np.array([x, y, z]) for x, y, z in earth_trajectory_normalized])
        )
        mars_path.set_points_as_corners(
            axes.coords_to_point([np.array([x, y, z]) for x, y, z in mars_trajectory_normalized])
        )

        # Time tracker
        time_tracker = ValueTracker(0)

        # Updaters for Earth and Mars
        def update_earth(mob):
            t = int(time_tracker.get_value()) % len(earth_trajectory_normalized)
            pos = earth_trajectory_normalized[t]
            mob.move_to(np.array([pos[0], pos[1], pos[2]]))

        def update_mars(mob):
            t = int(time_tracker.get_value()) % len(mars_trajectory_normalized)
            pos = mars_trajectory_normalized[t]
            mob.move_to(np.array([pos[0], pos[1], pos[2]]))

        earth.add_updater(update_earth)
        mars.add_updater(update_mars)

        # Animate the motion
        self.play(time_tracker.animate.set_value(len(earth_trajectory) - 1), run_time=20, rate_func=linear)

Manim Community v0.18.1

## Three-Body Problem

Once again, I used __ChatGPT__, but I had to fix the fact that it tried to make one of the bodies the sun, and it didn't move noticeably because it was so heavy and had no initial velocity.

In [67]:
%%manim -v WARNING -qm ChaoticThreeBodyScene

import rebound

# Constants
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
M1 = 1e30  # Mass of body 1 (Sun-like, kg)
M2 = 5.972e29  # Mass of body 2 (heavier Earth-like, kg)
M3 = 5.972e24  # Mass of body 3 (Earth-like, kg)

# Initial conditions for a chaotic three-body system
initial_conditions = [
    # Body 1
    [0, 0, 0, 0, -1e4, 0, M1],  # x, y, z, vx, vy, vz, mass
    # Body 2
    [1e11, 0, 0, 0, 1e4, 0, M2],  # 1 AU away, orbital velocity
    # Body 3
    [0, 1.2e11, 0, -2.9e4, 0, 0, M3],  # 1.2 AU away, perpendicular velocity
]

# Simulate the chaotic three-body problem using Rebound
def simulate_three_body_problem():
    sim = rebound.Simulation()
    sim.G = G
    sim.integrator = "ias15"  # High precision for chaotic systems

    # Add bodies
    for x, y, z, vx, vy, vz, m in initial_conditions:
        sim.add(m=m, x=x, y=y, z=z, vx=vx, vy=vy, vz=vz)

    # Simulation settings
    sim.dt = 1e5  # Timestep (seconds)
    t_span = 10 * 365.25 * 24 * 3600  # Simulate for 10 years
    t_steps = 1000  # Number of steps
    times = np.linspace(0, t_span, t_steps)

    # Store positions
    trajectories = {i: [] for i in range(len(sim.particles))}

    for t in times:
        sim.integrate(t)
        for i, p in enumerate(sim.particles):
            trajectories[i].append([p.x / 1e11, p.y / 1e11, p.z / 1e11])  # Normalize by 1 AU

    # Normalize all trajectories
    all_positions = np.vstack([np.array(trajectories[i]) for i in trajectories])
    max_extent = np.max(np.abs(all_positions))
    scale_factor = 1 / max_extent  # Scale to fit [-1, 1]
    for i in trajectories:
        trajectories[i] = np.array(trajectories[i]) * scale_factor

    return trajectories

# Run the simulation
trajectories = simulate_three_body_problem()

# Radii for visualization
R1 = 0.1  # Body 1 radius (scaled for visualization)
R2 = 0.05  # Body 2 radius (scaled for visualization)
R3 = 0.05  # Body 3 radius (scaled for visualization)


# Manim Scene
class ChaoticThreeBodyScene(ThreeDScene):
    def construct(self):
        # Set up 3D axes
        axes = ThreeDAxes(
            x_range=[-1, 1, 0.5],
            y_range=[-1, 1, 0.5],
            z_range=[-1, 1, 0.5],
            x_length=10,
            y_length=10,
            z_length=10,
        )
        axes.center()

        # Create three bodies
        body1 = Dot3D(radius=R1, color=YELLOW)
        body2 = Dot3D(radius=R2, color=BLUE)
        body3 = Dot3D(radius=R3, color=RED)

        # Add axes and bodies to the scene
        self.add(axes, body1, body2, body3)

        # Set the camera position
        self.set_camera_orientation(phi=75 * DEGREES, theta=45 * DEGREES, zoom=1)

        # Time tracker
        time_tracker = ValueTracker(0)

        # Define trajectories
        traj1 = trajectories[0]
        traj2 = trajectories[1]
        traj3 = trajectories[2]

        # Updaters for the bodies
        def update_body1(mob):
            t = int(time_tracker.get_value()) % len(traj1)
            pos = traj1[t]
            mob.move_to(axes.coords_to_point(pos[0], pos[1], pos[2]))

        def update_body2(mob):
            t = int(time_tracker.get_value()) % len(traj2)
            pos = traj2[t]
            mob.move_to(axes.coords_to_point(pos[0], pos[1], pos[2]))

        def update_body3(mob):
            t = int(time_tracker.get_value()) % len(traj3)
            pos = traj3[t]
            mob.move_to(axes.coords_to_point(pos[0], pos[1], pos[2]))

        body1.add_updater(update_body1)
        body2.add_updater(update_body2)
        body3.add_updater(update_body3)

        # Animate the motion
        self.play(time_tracker.animate.set_value(len(traj1) - 1), run_time=20, rate_func=linear)

Manim Community v0.18.1